In [1]:
from pycalphad import Database, Model, variables as v
from pycalphad.tests.datasets import ROSE_TDB

In [2]:
from pycalphad import equilibrium
import numpy as np

my_phases_rose = ['TEST']
comps = ['H', 'HE', 'LI', 'BE', 'B', 'C', 'N', 'O', 'F']
comps = sorted(comps)
conds = dict({v.T: 1000, v.P: 101325})
for comp in comps[1:]:
    conds[v.X(comp)] = 1.0/float(len(comps))
dbf = Database(ROSE_TDB)
eqx = equilibrium(Database(ROSE_TDB), comps, my_phases_rose, conds, calc_opts={'pdens': 10})

(1, 1, 458, 9)
(0, 0)


In [3]:
eqx.MU

<xarray.DataArray 'MU' (N: 1, P: 1, T: 1, X_BE: 1, X_C: 1, X_F: 1, X_H: 1, X_HE: 1, X_LI: 1, X_N: 1, X_O: 1, component: 9)>
array([[[[[[[[[[[[-5835.099176, -5835.099176, -5835.099176, -5835.099176,
                  -5835.099176, -5835.099177, -5835.099177, -5835.099176,
                  -5835.099177]]]]]]]]]]]])
Coordinates:
  * X_H        (X_H) float64 0.1111
  * component  (component) <U2 'B' 'BE' 'C' 'F' 'H' 'HE' 'LI' 'N' 'O'
  * X_LI       (X_LI) float64 0.1111
  * X_BE       (X_BE) float64 0.1111
  * T          (T) float64 1e+03
  * X_O        (X_O) float64 0.1111
  * N          (N) float64 1.0
  * X_N        (X_N) float64 0.1111
  * P          (P) float64 1.013e+05
  * X_C        (X_C) float64 0.1111
  * X_HE       (X_HE) float64 0.1111
  * X_F        (X_F) float64 0.1111

In [4]:
eqx.X

<xarray.DataArray 'X' (N: 1, P: 1, T: 1, X_BE: 1, X_C: 1, X_F: 1, X_H: 1, X_HE: 1, X_LI: 1, X_N: 1, X_O: 1, vertex: 9, component: 9)>
array([[[[[[[[[[[[[ 0.495627,  0.495627,  0.001249,  0.001249,  0.001249,
                    0.001249,  0.001249,  0.001249,  0.001249],
                  [ 0.001249,  0.001249,  0.495627,  0.001249,  0.001249,
                    0.001249,  0.495627,  0.001249,  0.001249],
                  [ 0.001249,  0.001249,  0.001249,  0.001249,  0.495627,
                    0.495627,  0.001249,  0.001249,  0.001249],
                  [ 0.001249,  0.001249,  0.001249,  0.495627,  0.001249,
                    0.001249,  0.001249,  0.495627,  0.001249],
                  [ 0.495627,  0.001249,  0.001249,  0.001249,  0.001249,
                    0.001249,  0.001249,  0.001249,  0.495627],
                  [ 0.001249,  0.495627,  0.001249,  0.001249,  0.001249,
                    0.001249,  0.001249,  0.001249,  0.495627],
                  [      nan,       na

In [5]:
from xarray import open_dataset

dev = open_dataset('global_grid_develop.cdf')
adv = open_dataset('global_grid_advcond.cdf')

In [6]:
dev.X

<xarray.DataArray 'X' (P: 1, T: 1, points: 458, component: 9)>
array([[[[ 1.     , ...,  0.     ],
         ..., 
         [ 0.22174, ...,  0.03227]]]])
Coordinates:
  * component  (component) object 'B' 'BE' 'C' 'F' 'H' 'HE' 'LI' 'N' 'O'
  * P          (P) float64 1.013e+05
  * T          (T) float64 1e+03
Dimensions without coordinates: points

In [7]:
(adv.X - dev.X).sel(points=-1)

<xarray.DataArray 'X' (N: 1, P: 1, T: 1, component: 9)>
array([[[[-0.221738, -0.048249, -0.166191,  0.903133,  0.00263 , -0.161201,
          -0.125674, -0.150441, -0.032269]]]])
Coordinates:
  * N          (N) float64 1.0
  * component  (component) object 'B' 'BE' 'C' 'F' 'H' 'HE' 'LI' 'N' 'O'
  * P          (P) float64 1.013e+05
  * T          (T) float64 1e+03

In [8]:
dev.sel(points=-1).X

<xarray.DataArray 'X' (P: 1, T: 1, component: 9)>
array([[[ 0.22174 ,  0.048249,  0.166201,  0.087078,  0.007142,  0.161201,
          0.125674,  0.150444,  0.03227 ]]])
Coordinates:
  * component  (component) object 'B' 'BE' 'C' 'F' 'H' 'HE' 'LI' 'N' 'O'
  * P          (P) float64 1.013e+05
  * T          (T) float64 1e+03

In [9]:
dev.sel(points=-1).Y

<xarray.DataArray 'Y' (P: 1, T: 1, internal_dof: 9)>
array([[[ 0.22174 ,  0.048249,  0.166201,  0.087078,  0.007142,  0.161201,
          0.125674,  0.150444,  0.03227 ]]])
Coordinates:
  * P        (P) float64 1.013e+05
  * T        (T) float64 1e+03
Dimensions without coordinates: internal_dof

In [10]:
adv.sel(points=-1).X

<xarray.DataArray 'X' (N: 1, P: 1, T: 1, component: 9)>
array([[[[  2.123154e-06,   9.672355e-08,   9.772629e-06,   9.902116e-01,
            9.772629e-03,   3.602385e-07,   2.516998e-07,   2.602736e-06,
            5.365673e-07]]]])
Coordinates:
  * N          (N) float64 1.0
  * component  (component) object 'B' 'BE' 'C' 'F' 'H' 'HE' 'LI' 'N' 'O'
  * P          (P) float64 1.013e+05
  * T          (T) float64 1e+03

In [11]:
adv.sel(points=-1).Y

<xarray.DataArray 'Y' (N: 1, P: 1, T: 1, internal_dof: 9)>
array([[[[ 0.22174 ,  0.048249,  0.166201,  0.087078,  0.007142,  0.161201,
           0.125674,  0.150444,  0.03227 ]]]])
Coordinates:
  * N        (N) float64 1.0
  * P        (P) float64 1.013e+05
  * T        (T) float64 1e+03
Dimensions without coordinates: internal_dof